In [1]:
# Extracción de Id, nombre y cantidad de facturas de cada cliente.
# Almacenamiento de la información extraída en una nueva tabla.

# Librerías a importar
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# Crear la sesión de spark
spark = SparkSession  \
    .builder \
    .appName("La cantidad facturas por cliente") \
    .config('spark.driver.extraClassPath', "mysql-connector-j-8.0.32.jar") \
    .getOrCreate()

# Propiedades para la conexión usando spark jdbc
url = "jdbc:mysql://localhost:3306/libreria"
propiedades = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "root",
    "password": "tooR"
}

In [3]:

# Leemos la tabla factura de la base de datos librería
factura_df = spark.read.jdbc(url=url, table="factura", properties=propiedades)

# Agrupamos la cantidad de facturas por el id del cliente
cliente_cantidad_factura_df = factura_df.groupBy("id_cliente").count()

# Leemos los datos de la tabla cliente
cliente_df = spark.read.jdbc(url=url, table="cliente", properties=propiedades)

# Unimos el dataframe cantidad_facturas_cliente_df con la tabla de cliente por medio del id del cliente
cliente_cantidad_factura_df = cliente_cantidad_factura_df.join(
    cliente_df,
    cliente_df.id == cliente_cantidad_factura_df.id_cliente)

# Seleccionamos solo la información necesaria: id del cliente, nombre, apellido, cantidad de facturas
cliente_cantidad_factura_df = cliente_cantidad_factura_df.select(
    col("id"),
    col("nombre"),
    col("apellido"),
    col("count").alias("cantidad_facturas")
).orderBy("cantidad_facturas", ascending=False)

# Mostramos el dataframe cantidad_facturas_cliente_df, verificamos con una vista previa.
cliente_cantidad_factura_df.show()

+---+---------+----------+-----------------+
| id|   nombre|  apellido|cantidad_facturas|
+---+---------+----------+-----------------+
| 60|   Jarrod|      Dach|                5|
| 46|   Forest|Jakubowski|                5|
| 27|     Avis|    Beatty|                4|
| 47|   Karlee|    Heller|                4|
| 16|    Frank|      Will|                4|
| 10|  Elliott|     Kulas|                4|
| 18|  Candice|      Hane|                4|
| 34|  Rodolfo|    Deckow|                3|
| 52|   Turner|    Sporer|                3|
| 13|    Aliya|     Marks|                3|
|  4|   Reggie|   Schmitt|                3|
| 23|    Ewald| VonRueden|                3|
| 39|  Christa|    Marvin|                3|
| 11|Cristobal|      Wiza|                3|
| 30|    Ewell|   Hackett|                3|
| 53|  Madison|    Schoen|                2|
| 45|   Jaiden|Balistreri|                2|
| 43| Emmanuel|Vandervort|                2|
| 38|    Zaria|Schowalter|                2|
| 36|     

In [ ]:
#Cargamos la informacion del dataframe cantidad_facturas_cliente_df en una tabla llamada cliente_cantidad_factura
cliente_cantidad_factura_df.write.jdbc(
    url=url, table="cliente_cantidad_factura", mode="overwrite", properties=propiedades)